## Validating BGP Route Policies with Batfish

Route policies for BGP are complex and error prone.  In this notebook we show how Batfish can be used to validate that your route policies behave as intended.  Batfish's `searchRoutePolicies` question allows you to easily explore the behavior of a route policy, as well as guarantee its correctness for *all possible* route announcements that it processes.

Let's start, as usual, by initializing the example network that we will use in this notebook.

![Analytics](https://ga-beacon.appspot.com/UA-100596389-3/open-source/pybatfish/jupyter_notebooks/validating-route-policies?pixel&useReferer)

In [1]:
# Import packages and load questions
%run startup.py
load_questions()

# Initialize a network and snapshot
NETWORK_NAME = "example_network"
SNAPSHOT_NAME = "example_snapshot"

SNAPSHOT_PATH = "networks/route-analysis"

bf_set_network(NETWORK_NAME)
bf_init_snapshot(SNAPSHOT_PATH, name=SNAPSHOT_NAME, overwrite=True)

'example_snapshot'

The network has two border routers, named `border1` and `border2`, which each have a BGP session with a router of another AS.  Our goal in this notebook is to validate that the out-bound route policy for this session, called `to_outside`, is correctly configured.

The intent of `to_outside` is as follows.  First, it should permit addresses that our network owns to be announced.  These are addresses in the range `1.2.3.[0-255]` and `1.2.4.[0-255]`.  Second, it should permit routes of our customers to be announced.  These are identified by announcements that have are tagged with a community that has AS number 40.  The policy should also set the MED for customer announcements to 50.  Finally, no other announcements should be allowed.

Ok, let's get started!

### Example 1: Announce Your Own Addresses


First let's ensure that our border routers both allow all of our own addresses to be announced.  Traditionally you might try to validate this policy by running one or more test packets, and as a first attempt we can try something like that with `searchRoutePolicies`:

In [2]:
# TODO: allow a single string as the prefix, instead of requiring a singleton list
inRoute = BgpRouteConstraints(prefix=["1.2.3.4/32"])
bfq.searchRoutePolicies(policies="to_outside", inputConstraints=inRoute).answer().frame()

,Node,Policy_Name,Input_Route,Action,Output_Route
0,border1,to_outside,"BgpRoute(network='1.2.3.4/32', originatorIp='0.0.0.0', originType='igp', protocol='bgp', asPath=[], communities=[], localPreference=0, metric=0, sourceProtocol=None)",PERMIT,"BgpRoute(network='1.2.3.4/32', originatorIp='0.0.0.0', originType='igp', protocol='bgp', asPath=[], communities=[], localPreference=0, metric=0, sourceProtocol=None)"
1,border2,to_outside,"BgpRoute(network='1.2.3.4/32', originatorIp='0.0.0.0', originType='igp', protocol='bgp', asPath=[], communities=[], localPreference=0, metric=0, sourceProtocol=None)",PERMIT,"BgpRoute(network='1.2.3.4/32', originatorIp='0.0.0.0', originType='igp', protocol='bgp', asPath=[], communities=[], localPreference=0, metric=0, sourceProtocol=None)"


The first line of code above creates a `BgpRouteConstraints` object that represents requirements on the input announcement, in this case just that it announces the prefix 1.2.3.4/32.  The second line asks the `searchRoutePolicies` question to find an example of a route announcement meeting our constraints that is permitted by `to_outside`.  (In general the policies of interest can be specified as a regular expression, as can the set of nodes to consider.)

The output of the question is an example behavior of the `to_outside` route policy for each border router.  It shows that a route announcement with the specified prefix, along with simple concrete values for other fields of the announcement, will be permitted by both route policies and output unchanged.

That's nice, but it is just a single example.  We'd like to know that *all* of the addresses that we own will be permitted by the route maps.  To do that, we will change our perspective a bit.  Instead of asking for a "good" example, which satisfies our intended policy, we will ask `searchRoutePolicies` to search for a "bad" example, which violates our policy.  If this results in one or more results, then our configuration is buggy.  If this results in no results, then we can be sure that our configuration satisfies the policy, for all possible route announcements.

In [3]:
ownedSpace = ["1.2.3.0/24:24-32", "1.2.4.0/24:24-32"]
inRoute = BgpRouteConstraints(prefix=ownedSpace)
bfq.searchRoutePolicies(policies="to_outside", inputConstraints=inRoute, action="deny").answer().frame()

,Node,Policy_Name,Input_Route,Action,Output_Route


The code above has two differences from our earlier version.  First, we specify a set of prefixes rather than a single one.  In this case, our input announcement can announce any prefix in the range 1.2.3.0/24:24-32 or 1.2.4.0/24:24-32.  Second, we ask for an example behavior of `to_outside` that *denies* the announcement via the `action` parameter (whose default value is `permit`).

As a result, this call to `searchRoutePolicies` asks:  Is there  any announcement for a prefix in the range 1.2.3.0/24:24-32 or 1.2.4.0/24:24-32 that is denied by `to_outside`?  Since there are no results, this implies that no such announcement exists, and therefore that our policy holds, for both `border1` and `border2`.  Hurray!

### Example 2: Announce Your Customer's Routes

Next we will check that the `to_outside` route policies are properly announcing our customers' routes.  First we can use the same approach that we used above to search for a violation, which in this case is an announcement tagged with a community that has AS 40 but is denied:

In [4]:
customerCommunities = "40:.*"
# TODO: allow a single string for a community regex, instead of requiring a singleton list
inRoute = BgpRouteConstraints(communities=[customerCommunities])
bfq.searchRoutePolicies(policies="to_outside", inputConstraints=inRoute, action="deny").answer().frame()

,Node,Policy_Name,Input_Route,Action,Output_Route


There are no answers, so we can be sure that all of our customer routes are being permitted.  But we're not done yet!  We would also like to check that these customer announcements are properly transformed, in particular to have a MED value of 50.  To do that, we will impose constraints not only on the input route announcement to `to_outside`, as we have been doing so far, but also on the output route announcement that is produced when an announcement is permitted.  

In this case, we will ask the following:  Is there a permitted customer route announcement whose output announcement does *not* have a MED value of 50?  Here's how it's done:

In [5]:
inRoute = BgpRouteConstraints(communities=[customerCommunities])
# TODO: make integer spaces take a list instead of a single string, so it's uniform with other fields
wrongMed = "0-255, !50"
outRoute = BgpRouteConstraints(med=wrongMed)
bfq.searchRoutePolicies(policies="to_outside", inputConstraints=inRoute, outputConstraints=outRoute).answer().frame()

,Node,Policy_Name,Input_Route,Action,Output_Route
0,border1,to_outside,"BgpRoute(network='0.0.0.0/0', originatorIp='0.0.0.0', originType='igp', protocol='bgp', asPath=[], communities=['40:0'], localPreference=0, metric=0, sourceProtocol=None)",PERMIT,"BgpRoute(network='0.0.0.0/0', originatorIp='0.0.0.0', originType='igp', protocol='bgp', asPath=[], communities=['40:0'], localPreference=0, metric=0, sourceProtocol=None)"


The `wrongMed` variable above specifies the MED values of interest:  everything in the range 0-255 except for 50.  We then constrain output announcements to have a MED value in that set.  The output tells us two important things.  First, because there is no example for `border2`, its `to_outside` route policy satisfies our intent:  all customer announcements are output with a MED value of 50.  Second, the result for `border1` concretely illustrates that its version of the `to_outside` route policy violates our intent.  Inspecting the `to_outside` route policy for `border1` reveals the problem, namely that the clause pertaining to customer announcements accidentally omits the necessary `set metric` command.

### Example 3: Filter All Other Routes

Finally, we want to make sure that our `to_outside` route policies don't announce any routes other than those that we own and those of our customers.  As you've figured out by now, the way to do that is to ask `searchRoutePolicies` to find a violation, which in this case is an accepted announcement that is neither for a prefix we own nor having the appropriate community tag for customers.

In [6]:
inRoute = BgpRouteConstraints(prefix=ownedSpace, complementPrefix=True,
                              communities=[customerCommunities], complementCommunities=True)
bfq.searchRoutePolicies(policies="to_outside", inputConstraints=inRoute).answer().frame()

,Node,Policy_Name,Input_Route,Action,Output_Route
0,border2,to_outside,"BgpRoute(network='1.2.2.0/23', originatorIp='0.0.0.0', originType='igp', protocol='bgp', asPath=[], communities=[], localPreference=0, metric=0, sourceProtocol=None)",PERMIT,"BgpRoute(network='1.2.2.0/23', originatorIp='0.0.0.0', originType='igp', protocol='bgp', asPath=[], communities=[], localPreference=0, metric=0, sourceProtocol=None)"


Above we use two parameters to `searchRoutePolicies` that you haven't seen yet:  the `complementPrefix` flag indicates that we are interested in prefixes outside the set specified by the `prefix` parameter, and the `complementCommunities` flag does the same thing for the `communities` parameter.  There are no results for `border1`'s `to_outside` policy, so Batfish has proven that the policy does not announce any routes other than those that either we own or are announced by our customers.  However, Batfish reports a violation for `border2`:  its route policy permits announcements for the prefix 1.2.2.0/23 when no communities are present.  A look at the configuration for that router identifies the problem:  its prefix list named `our-networks` accidentally allows prefixes of 1.2.3.0 with lengths 23 and higher instead of 24 and higher.

### Summary


Some summary text here!

Want to learn more? Come find us on [Slack](https://join.slack.com/t/batfish-org/shared_invite/enQtMzA0Nzg2OTAzNzQ1LTUxOTJlY2YyNTVlNGQ3MTJkOTIwZT     U2YjY3YzRjZWFiYzE4ODE5ODZiNjA4NGI5NTJhZmU2ZTllOTMwZDhjMzA) and [GitHub](https://github.com/batfish/batfish)